# V25: Localization consistency across sectors (82 vs 83)

Goal: check whether pixel-level localization evidence is **consistent across independent sectors**.

This notebook re-runs:
- V08 centroid shift, and
- V09 difference image

for sectors **82** and **83**, then renders side-by-side plots.


In [ ]:
from __future__ import annotations

import json
import sys
from pathlib import Path

import numpy as np

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh
import bittr_tess_vetter.api as btv

step_id = '39_v25_localization_consistency_82_83'
run_out_dir, docs_out_dir = sh.artifact_dirs(step_id=step_id)

ds = sh.load_dataset()

# Baseline candidate depth from stitched PDCSAP (for mask windows)
stitched = sh.stitch_pdcsap(ds)
depth_ppm, _ = sh.estimate_depth_ppm(stitched)
candidate = sh.make_candidate(depth_ppm)

sectors = [82, 83]

# Load per-sector TPF stamps (download missing sectors once, then cache on disk)
tpf_by_sector: dict[int, btv.TPFStamp] = {int(k): v for k, v in ds.tpf_by_sector.items()}
cache_dir = Path('persistent_cache/tutorial_toi-5807-incremental/tpfs')
cache_dir.mkdir(parents=True, exist_ok=True)

missing = [int(s) for s in sectors if int(s) not in tpf_by_sector]
if missing:
    import lightkurve as lk

    for sector in missing:
        npz = cache_dir / f'sector{sector}_tpf.npz'
        if npz.exists():
            d = np.load(npz, allow_pickle=True)
            try:
                from astropy.wcs import WCS

                wcs = WCS(d['wcs_header'].item()) if 'wcs_header' in d else None
            except Exception:
                wcs = d['wcs_header'].item() if 'wcs_header' in d else None

            tpf_by_sector[sector] = btv.TPFStamp(
                time=np.asarray(d['time'], dtype=np.float64),
                flux=np.asarray(d['flux'], dtype=np.float64),
                flux_err=np.asarray(d['flux_err'], dtype=np.float64) if 'flux_err' in d else None,
                wcs=wcs,
                aperture_mask=np.asarray(d['aperture_mask'], dtype=bool) if 'aperture_mask' in d else None,
                quality=np.asarray(d['quality'], dtype=np.int32) if 'quality' in d else None,
            )
            continue

        search = lk.search_targetpixelfile(f'TIC {sh.TIC_ID}', sector=int(sector), exptime=120)
        tpf = search.download() if len(search) else None
        if tpf is None:
            raise RuntimeError(f'No TPF available for sector {sector}')

        stamp = btv.TPFStamp(
            time=np.asarray(tpf.time.value, dtype=np.float64),
            flux=np.asarray(tpf.flux.value, dtype=np.float64),
            flux_err=np.asarray(tpf.flux_err.value, dtype=np.float64),
            wcs=tpf.wcs,
            aperture_mask=np.asarray(tpf.pipeline_mask, dtype=bool),
            quality=np.asarray(tpf.quality, dtype=np.int32),
        )
        tpf_by_sector[sector] = stamp

        # Cache to disk (best-effort)
        try:
            wcs_header = dict(tpf.wcs.to_header()) if getattr(tpf, 'wcs', None) is not None else None
            np.savez(
                npz,
                time=stamp.time,
                flux=stamp.flux,
                flux_err=stamp.flux_err,
                wcs_header=wcs_header,
                aperture_mask=stamp.aperture_mask,
                quality=stamp.quality,
            )
        except Exception:
            pass

results: dict[str, dict] = {}
r08_by_sector = {}
r09_by_sector = {}
for sector in sectors:
    lc0 = ds.lc_by_sector[int(sector)]
    q = np.asarray(
        lc0.quality if getattr(lc0, 'quality', None) is not None else np.zeros(len(lc0.time)),
        dtype=np.int32,
    )
    lc_sec = btv.LightCurve(time=lc0.time, flux=lc0.flux, flux_err=lc0.flux_err, quality=q)
    tpf = tpf_by_sector[int(sector)]

    session = btv.VettingSession.from_api(
        lc=lc_sec,
        candidate=candidate,
        stellar=sh.STELLAR,
        tpf=tpf,
        network=False,
        tic_id=sh.TIC_ID,
    )

    r08 = session.run('V08')
    r09 = session.run('V09')
    r08_by_sector[int(sector)] = r08
    r09_by_sector[int(sector)] = r09

    def _pick(metrics: dict, keys: list[str]) -> dict:
        return {k: metrics.get(k) for k in keys}

    results[str(sector)] = {
        'V08': {
            'status': r08.status,
            'flags': r08.flags,
            'metrics': _pick(
                dict(r08.metrics),
                [
                    'centroid_shift_pixels',
                    'shift_uncertainty_pixels',
                    'significance_sigma',
                    'centroid_shift_arcsec',
                    'n_in_transit_cadences',
                    'n_out_of_transit_cadences',
                ],
            ),
            'plot_data': (r08.raw or {}).get('plot_data', {}),
        },
        'V09': {
            'status': r09.status,
            'flags': r09.flags,
            'metrics': _pick(
                dict(r09.metrics),
                [
                    'max_depth_pixel_offset',
                    'max_depth_pixel_distance',
                    'aperture_sum_ppm',
                ],
            ),
            'plot_data': (r09.raw or {}).get('plot_data', {}),
        },
    }

json_name = 'localization_consistency_82_83.json'
(run_out_dir / json_name).write_text(json.dumps(results, indent=2, sort_keys=True))
if docs_out_dir is not None:
    (docs_out_dir / json_name).write_text(json.dumps(results, indent=2, sort_keys=True))

print(json.dumps(results, indent=2, sort_keys=True)[:2000] + '\n...')


<details>
<summary><b>Expected Output</b></summary>

A per-sector JSON summary is written to:
- `docs/tutorials/artifacts/tutorial_toi-5807-incremental/39_v25_localization_consistency_82_83/localization_consistency_82_83.json`

```text
{
  "82": {
    "V08": {
      "centroid_shift_arcsec": 0.09029882097528287,
      "centroid_shift_pixels": 0.004299943855965851,
      "n_in_transit_cadences": 243,
      "n_out_of_transit_cadences": 17648,
      "shift_uncertainty_pixels": 0.0027261292100679087,
      "significance_sigma": 1.5773074291877522
    },
    "V08_flags": [],
    "V09": {
      "aperture_sum_ppm": null,
      "max_depth_pixel_distance": null,
      "max_depth_pixel_offset": null
    },
    "V09_flags": [
      "DIFFIMG_MAX_AT_EDGE",
      "DIFFIMG_TARGET_DEPTH_NONPOSITIVE",
      "DIFFIMG_UNRELIABLE"
    ]
  },
  "83": {
    "V08": {
      "centroid_shift_arcsec": 0.35706258426488147,
      "centroid_shift_pixels": 0.017002980203089595,
      "n_in_transit_cadences": 244,
      "n_out_of_transit_cadences": 16841,
      "shift_uncertainty_pixels": 0.0051968726000012075,
      "significance_sigma": 3.2717716041539373
    },
    "V08_flags": [],
    "V09": {
      "aperture_sum_ppm": null,
      "max_depth_pixel_distance": null,
      "max_depth_pixel_offset": null
    },
    "V09_flags": [
      "DIFFIMG_MAX_AT_EDGE",
      "DIFFIMG_TARGET_DEPTH_NONPOSITIVE",
      "DIFFIMG_UNRELIABLE"
    ]
  }
}
```

</details>


In [ ]:
import json
from pathlib import Path

import matplotlib.pyplot as plt

from bittr_tess_vetter.api import plot_centroid_shift, plot_difference_image

step_id = '39_v25_localization_consistency_82_83'
run_out_dir, docs_out_dir = sh.artifact_dirs(step_id=step_id)

fig, axes = plt.subplots(2, 2, figsize=(12.5, 9.0), dpi=150)

for j, sector in enumerate([82, 83]):
    # V08
    v08 = r08_by_sector[int(sector)]
    ax = axes[0, j]
    ax.set_title(f'V08 Centroid shift (sector {sector})')
    try:
        plot_centroid_shift(v08, ax=ax, show_colorbar=False)
    except Exception as e:
        ax.text(0.5, 0.5, f'plot failed: {e}', ha='center', va='center')

    # V09
    v09 = r09_by_sector[int(sector)]
    ax = axes[1, j]
    ax.set_title(f'V09 Difference image (sector {sector})')
    try:
        plot_difference_image(v09, ax=ax, show_colorbar=False)
    except Exception as e:
        ax.text(0.5, 0.5, f'plot failed: {e}', ha='center', va='center')

fig.tight_layout()

png_name = 'V25_localization_consistency_82_83.png'
run_png = run_out_dir / png_name
fig.savefig(run_png)
if docs_out_dir is not None:
    fig.savefig(docs_out_dir / png_name)
plt.close(fig)

print(json.dumps({'run_plot_path': str(run_png), 'docs_plot_path': str((docs_out_dir / png_name) if docs_out_dir is not None else None)}, indent=2, sort_keys=True))


## Plot

<img src="../artifacts/tutorial_toi-5807-incremental/39_v25_localization_consistency_82_83/V25_localization_consistency_82_83.png" width="980" />


<details>
<summary><b>Analysis</b></summary>

- **What to look for:** the centroid/difference-image evidence should point to a consistent location across sectors.
- **Interpretation:**
  - If sector 82 and 83 localize to the same pixel neighborhood and show similar centroid behavior, that supports an on-target (or at least stable-host) interpretation.
  - If one sector localizes off-target while the other does not, treat this as a serious warning (blend/systematics risk) and gate further analysis accordingly.
- **Caveats:** these are noisy diagnostics at low SNR; consistency is more informative than any single-sector metric.

</details>
